# SPARQL

This document contains a short introduction to [SPARQL](https://www.w3.org/TR/sparql11-query/) using [rudof](https://rudof-project.github.io/).


## Preliminaries

The library is available as `pyrudof`.

In [1]:
!pip install pyrudof

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 75.4 MB/s eta 0:00:00


The main entry point if a class called `Rudof` through which most of the functionality is provided.

In [2]:
from pyrudof import Rudof, RudofConfig

In order to initialize that class, it is possible to pass a RudofConfig instance which contains configuration parameters for customization.

In [3]:
rudof = Rudof(RudofConfig())

## SPARQL queries with local RDF data

As SPARQL is an RDF query language, we first need some RDF data.

rudof can run queries against local RDF data or RDF that is available through some SPARQL endpoint.

Let's start with some local RDF data.

In [4]:
rdf = """
prefix :    <http://example.org/>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>

:alice a :Person ;
 :name      "Alice"                ;
 :birthDate "2005-03-01"^^xsd:date ;
 :worksFor  :acme                   .
:bob a :Person   ;
 :name      "Robert Smith"         ;
 :birthDate "2003-01-02"^^xsd:date ;
 :worksFor  :acme  .
:acme a :Company ;
 :name "Acme Inc." .
"""
rudof.read_data_str(rdf)

We can run SPARQL queries as follows:

In [5]:
query = """
PREFIX : <http://example.org/>

SELECT ?person ?name ?date ?company WHERE {
  ?person a          :Person ;
          :name      ?name   ;
          :birthDate ?date   ;
          :worksFor  ?c   .
  ?c      :name      ?company .
}
"""

results = rudof.run_query_str(query)


Show the results:

In [6]:
print(results.show())

╭───┬─────────┬────────────────┬────────────────────────┬─────────────╮
│   │ ?person │ ?name          │ ?date                  │ ?company    │
├───┼─────────┼────────────────┼────────────────────────┼─────────────┤
│ 1 │ :alice  │ "Alice"        │ "2005-03-01"^^xsd:date │ "Acme Inc." │
├───┼─────────┼────────────────┼────────────────────────┼─────────────┤
│ 2 │ :bob    │ "Robert Smith" │ "2003-01-02"^^xsd:date │ "Acme Inc." │
╰───┴─────────┴────────────────┴────────────────────────┴─────────────╯



In [7]:
rudof.reset_all()

## SPARQL queries with an endpoint

It is also possible to run SPARQL queries against SPARQL endpoints.

SPARQL endpoints are usually identified by an IRI.

rudof contains a list of popular SPARQL endpoints which can also be identified by name, like wikidata, whose IRI is: https://query.wikidata.org/sparql

In [8]:
query = """
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
SELECT ?person ?occupation WHERE {
    ?p wdt:P31 wd:Q5 ;
          wdt:P106 ?o ;
          rdfs:label ?person ;
          wdt:P19 wd:Q14317 .
  ?o rdfs:label ?occupation
  FILTER (lang(?person) = "en" && lang(?occupation) = "en")
}
LIMIT 10
"""

In [9]:
rudof.use_endpoint("https://query.wikidata.org/sparql")


In [10]:
results = rudof.run_query_str(query)

In [11]:
print(results.show())

╭────┬──────────────────────────────────────────┬────────────────────────╮
│    │ ?person                                  │ ?occupation            │
├────┼──────────────────────────────────────────┼────────────────────────┤
│ 1  │ "Luis López-Doriga"en                    │ "politician"en         │
├────┼──────────────────────────────────────────┼────────────────────────┤
│ 2  │ "Leticia Sánchez Ruiz"en                 │ "writer"en             │
├────┼──────────────────────────────────────────┼────────────────────────┤
│ 3  │ "Luis Menéndez-Pidal y Álvarez"en        │ "architect"en          │
├────┼──────────────────────────────────────────┼────────────────────────┤
│ 4  │ "Elena Fernández"en                      │ "actor"en              │
├────┼──────────────────────────────────────────┼────────────────────────┤
│ 5  │ "Alberto Rionda"en                       │ "composer"en           │
├────┼──────────────────────────────────────────┼────────────────────────┤
│ 6  │ "Fernando Alonso"e

In [12]:
rudof.use_endpoint("https://query.wikidata.org/sparql")


In [13]:
rudof.reset_all()

## Federated queries

TBD

## CONSTRUCT queries

In [18]:
from pyrudof import QueryResultFormat

It is also possible to run SPARQL CONSTRUCT queries which can be used to return RDF results.

For example:

In [19]:
query = """
PREFIX wd:  <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX :    <http://example.org/>

CONSTRUCT {
   ?p a     :Person ;
      :name ?person ;
      :occupation ?occupation
} WHERE {
    ?p wdt:P31 wd:Q5 ;
          wdt:P106 ?o ;
          rdfs:label ?person ;
          wdt:P19 wd:Q14317 .
  ?o rdfs:label ?occupation
  FILTER (lang(?person) = "en" && lang(?occupation) = "en")
}
LIMIT 10
"""

We indicate rudof that we will use Wikidata's endpoint:

In [20]:
rudof.use_endpoint("https://query.wikidata.org/sparql")

In [21]:
result = rudof.run_query_construct_str(query, QueryResultFormat.Turtle)

In [22]:
print(result)

@prefix wd: <http://www.wikidata.org/entity/> .
@prefix wdt: <http://www.wikidata.org/prop/direct/> .
@prefix : <http://example.org/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix sesame: <http://www.openrdf.org/schema/sesame#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix fn: <http://www.w3.org/2005/xpath-functions#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix dc: <http://purl.org/dc/elements/1.1/> .
@prefix hint: <http://www.bigdata.com/queryHints#> .
@prefix bd: <http://www.bigdata.com/rdf#> .
@prefix bds: <http://www.bigdata.com/rdf/search#> .
@prefix psn: <http://www.wikidata.org/prop/statement/value-normalized/> .
@prefix pqn: <http://www.wikidata.org/prop/qualifier/value-normalized/> .
@prefix prn: <http://www.wikidata.org/prop/reference/value-normalized/> .
@prefix mwapi: <https://www.mediawiki.org/ontology#API/> .
@prefix ga

## References



### Resources for learning SPARQL

*   [Learning SPARQL](https://learningsparql.com/) book and blog by Bob Ducharne.
*   [SPARQL tutorial](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial) from Wikidata

### Lists of SPARQL endpoints

*   [SPARQL endpoints](https://www.wikidata.org/wiki/Wikidata:Lists/SPARQL_endpoints) collected in Wikidata
*   [SPARQL endpoints](https://www.w3.org/wiki/SparqlEndpoints) collected at W3C.

